## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Belaya Gora,RU,68.53,146.42,-3.37,92,99,1.59,overcast clouds
1,1,Hobart,AU,-42.88,147.33,66.00,44,20,13.87,few clouds
2,2,Swellendam,ZA,-34.02,20.44,59.00,77,5,8.05,clear sky
3,3,Albany,US,42.60,-73.97,51.01,93,75,2.98,broken clouds
4,4,Noumea,NC,-22.28,166.46,80.60,54,90,13.87,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Noumea,NC,-22.28,166.46,80.60,54,90,13.87,overcast clouds
7,7,Rikitea,PF,-23.12,-134.97,71.69,75,100,19.46,overcast clouds
13,13,Sao Filipe,CV,14.90,-24.50,79.14,75,87,7.43,overcast clouds
16,16,Bouna,CI,9.27,-3.00,72.82,82,100,4.29,overcast clouds
19,19,Ayorou,NE,14.73,0.92,75.42,18,0,9.91,clear sky
20,20,El Tigre,VE,8.89,-64.25,76.60,85,43,8.95,scattered clouds
23,23,Saint-Philippe,RE,-21.36,55.77,80.60,65,0,17.22,clear sky
29,29,Yashan,CN,22.20,109.94,78.73,50,0,12.93,clear sky
34,34,Kilindoni,TZ,-7.91,39.67,79.97,82,29,13.02,moderate rain
37,37,Arraial Do Cabo,BR,-22.97,-42.03,71.96,84,95,8.46,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Noumea,NC,80.60,overcast clouds,-22.28,166.46,
7,Rikitea,PF,71.69,overcast clouds,-23.12,-134.97,
13,Sao Filipe,CV,79.14,overcast clouds,14.90,-24.50,
16,Bouna,CI,72.82,overcast clouds,9.27,-3.00,
19,Ayorou,NE,75.42,clear sky,14.73,0.92,
20,El Tigre,VE,76.60,scattered clouds,8.89,-64.25,
23,Saint-Philippe,RE,80.60,clear sky,-21.36,55.77,
29,Yashan,CN,78.73,clear sky,22.20,109.94,
34,Kilindoni,TZ,79.97,moderate rain,-7.91,39.67,
37,Arraial Do Cabo,BR,71.96,overcast clouds,-22.97,-42.03,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Noumea,NC,80.60,overcast clouds,-22.28,166.46,Hôtel Le Lagon
7,Rikitea,PF,71.69,overcast clouds,-23.12,-134.97,Pension Maro'i
13,Sao Filipe,CV,79.14,overcast clouds,14.90,-24.50,Tortuga B&B
16,Bouna,CI,72.82,overcast clouds,9.27,-3.00,Hotel Reserve
19,Ayorou,NE,75.42,clear sky,14.73,0.92,Hotel Amenokal


In [12]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and </dt><dd>{Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))